In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.float_format', '{:.2g}'.format)

10 instâncias com 2, 5 e 10 ingredientes.

1 ou 2 produtos finais.

Instâncias LLL e HHH.

Cap alta e baixa.

Total de instâncias: 10\*3\*2\*2\*2 = 240

In [3]:
df_formulacao1 = pd.read_excel("SOLUTION/2024 04 23 FINAL/formulacao1.xlsx", engine="openpyxl")
df_variaveis = pd.read_excel("SOLUTION/2024 04 23 FINAL/variaveis.xlsx", engine="openpyxl")

In [4]:
df_formulacao1.head()

,setup_cost_end_products,production_cost_end_products,holding_cost_end_products,end_products_cost,setup_cost_ingredients,production_cost_ingredients,holding_cost_end_ingredients,ingredients_cost,objective_function,Instance,Best Bound,Gap,Nodes Processed,Tempo de Solução,capacity,amount_of_end_products,Relaxed Objective Value
0,5.3e+05,0,3.7e+04,5.6e+05,6.3e+05,1.5e+07,2.9e+05,1.6e+07,1.7e+07,5HHH2,1.7e+07,9.9e-05,9.9e-05,2,6.6e+03,1,1.7e+07
1,1.3e+05,0,2.3e+03,1.3e+05,1.2e+04,1.3e+06,6e+03,1.3e+06,1.4e+06,5LLL1,1.4e+06,0.0013,0.0013,3.6e+03,1.5e+03,2,1.4e+06
2,3e+04,0,2e+03,3.2e+04,5.8e+03,4.8e+05,2.7e+03,4.8e+05,5.2e+05,2LLL4,5.2e+05,0.0001,0.0001,11,7.9e+03,1,5.2e+05
3,1.5e+06,0,2.3e+04,1.5e+06,4.5e+05,1.5e+07,6.6e+05,1.6e+07,1.8e+07,10HHH10,1.8e+07,6.4e-05,6.4e-05,0.12,6.4e+03,1,1.8e+07
4,1.1e+06,0,8.4e+04,1.1e+06,5.5e+05,2.8e+07,5e+05,2.9e+07,3e+07,10HHH2,3e+07,0.00011,0.00011,3.6e+03,6.8e+03,2,3e+07


Todas as instâncias estão com gaps < 1%.

In [5]:
df_formulacao1[["Gap"]].describe()

,Gap
count,1.2e+02
mean,0.00041
std,0.00054
min,0
25%,9.9e-05
50%,0.0001
75%,0.00062
max,0.0027


- Em média % em relação a função objetivo, o maior custo vem do custo de produção dos ingredientes (91% +- 2,7%)
- Em média % em relação a fob, o custo de estoque e custo de setup dos ingredientes está pareado (setup: 1,7% +- 1% e estoque: 1,4% +- 1%)

In [6]:
df_formulacao1["setup_cost_end_products_perc"] = df_formulacao1["setup_cost_end_products"]/df_formulacao1["objective_function"]
df_formulacao1["holding_cost_end_products_perc"] = df_formulacao1["holding_cost_end_products"]/df_formulacao1["objective_function"]
df_formulacao1["end_products_cost_perc"] = df_formulacao1["end_products_cost"]/df_formulacao1["objective_function"]
df_formulacao1["setup_cost_ingredients_perc"] = df_formulacao1["setup_cost_ingredients"]/df_formulacao1["objective_function"]
df_formulacao1["production_cost_ingredients_perc"] = df_formulacao1["production_cost_ingredients"]/df_formulacao1["objective_function"]
df_formulacao1["holding_cost_end_ingredients_perc"] = df_formulacao1["holding_cost_end_ingredients"]/df_formulacao1["objective_function"]
df_formulacao1["ingredients_cost_perc"] = df_formulacao1["ingredients_cost"]/df_formulacao1["objective_function"]

df_formulacao1["objective_function_without_ingredients"] = df_formulacao1["objective_function"] - df_formulacao1["production_cost_ingredients"]

cols1 = ["setup_cost_end_products_perc", "holding_cost_end_products_perc", "setup_cost_ingredients_perc", "production_cost_ingredients_perc", "holding_cost_end_ingredients_perc"]

df_formulacao1[cols1].describe()

,setup_cost_end_products_perc,holding_cost_end_products_perc,setup_cost_ingredients_perc,production_cost_ingredients_perc,holding_cost_end_ingredients_perc
count,1.2e+02,1.2e+02,1.2e+02,1.2e+02,1.2e+02
mean,0.058,0.0031,0.017,0.91,0.014
std,0.018,0.0025,0.01,0.027,0.011
min,0.031,0.00095,0.0056,0.84,0.0025
25%,0.046,0.0019,0.0086,0.89,0.0058
50%,0.057,0.0025,0.018,0.91,0.0092
75%,0.064,0.0033,0.025,0.93,0.017
max,0.11,0.019,0.039,0.96,0.038


Retirando o custo de produção dos ingredientes da fob:
- O equilíbrio entre o custo de setup e estoque se mantém (setup: 18% +- 8%, estoque: 14% +- 8%)
- O setup dos produtos finais tem a maior participação % (64% +- 14%)

In [7]:
df_formulacao1["setup_cost_end_products_perc2"] = df_formulacao1["setup_cost_end_products"]/df_formulacao1["objective_function_without_ingredients"]
df_formulacao1["holding_cost_end_products_perc2"] = df_formulacao1["holding_cost_end_products"]/df_formulacao1["objective_function_without_ingredients"]
df_formulacao1["end_products_cost_perc2"] = df_formulacao1["end_products_cost"]/df_formulacao1["objective_function_without_ingredients"]
df_formulacao1["setup_cost_ingredients_perc2"] = df_formulacao1["setup_cost_ingredients"]/df_formulacao1["objective_function_without_ingredients"]
df_formulacao1["production_cost_ingredients_perc2"] = df_formulacao1["production_cost_ingredients"]/df_formulacao1["objective_function_without_ingredients"]
df_formulacao1["holding_cost_end_ingredients_perc2"] = df_formulacao1["holding_cost_end_ingredients"]/df_formulacao1["objective_function_without_ingredients"]

cols2 = ["setup_cost_end_products_perc2", "holding_cost_end_products_perc2", "setup_cost_ingredients_perc2", "holding_cost_end_ingredients_perc2"]

df_formulacao1[cols2].describe()

,setup_cost_end_products_perc2,holding_cost_end_products_perc2,setup_cost_ingredients_perc2,holding_cost_end_ingredients_perc2
count,1.2e+02,1.2e+02,1.2e+02,1.2e+02
mean,0.64,0.037,0.18,0.14
std,0.14,0.026,0.084,0.08
min,0.35,0.0074,0.059,0.041
25%,0.51,0.018,0.12,0.072
50%,0.66,0.031,0.16,0.11
75%,0.76,0.048,0.23,0.18
max,0.88,0.16,0.42,0.34


Selecionando uma instância em que o % de custo de estoque é menor que o % do custo de setup. Ou seja, procurando entender se gastamos mais com setup do que com estoque.

Escolhido: 5HHH2

In [8]:
df_formulacao1.query("holding_cost_end_products_perc2 < setup_cost_ingredients_perc2")

,setup_cost_end_products,production_cost_end_products,holding_cost_end_products,end_products_cost,setup_cost_ingredients,production_cost_ingredients,holding_cost_end_ingredients,ingredients_cost,objective_function,Instance,...,production_cost_ingredients_perc,holding_cost_end_ingredients_perc,ingredients_cost_perc,objective_function_without_ingredients,setup_cost_end_products_perc2,holding_cost_end_products_perc2,end_products_cost_perc2,setup_cost_ingredients_perc2,production_cost_ingredients_perc2,holding_cost_end_ingredients_perc2
0,5.3e+05,0,3.7e+04,5.6e+05,6.3e+05,1.5e+07,2.9e+05,1.6e+07,1.7e+07,5HHH2,...,0.91,0.017,0.97,1.5e+06,0.35,0.025,0.38,0.42,10,0.2
1,1.3e+05,0,2.3e+03,1.3e+05,1.2e+04,1.3e+06,6e+03,1.3e+06,1.4e+06,5LLL1,...,0.9,0.0042,0.91,1.5e+05,0.86,0.016,0.88,0.081,8.7,0.041
2,3e+04,0,2e+03,3.2e+04,5.8e+03,4.8e+05,2.7e+03,4.8e+05,5.2e+05,2LLL4,...,0.92,0.0051,0.94,4.1e+04,0.74,0.049,0.79,0.14,12,0.065
3,1.5e+06,0,2.3e+04,1.5e+06,4.5e+05,1.5e+07,6.6e+05,1.6e+07,1.8e+07,10HHH10,...,0.85,0.037,0.91,2.6e+06,0.57,0.0087,0.58,0.17,5.8,0.25
4,1.1e+06,0,8.4e+04,1.1e+06,5.5e+05,2.8e+07,5e+05,2.9e+07,3e+07,10HHH2,...,0.93,0.016,0.96,2.2e+06,0.48,0.038,0.52,0.25,13,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1.1e+06,0,3.2e+04,1.1e+06,5.3e+05,1.4e+07,6.1e+05,1.6e+07,1.7e+07,2HHH8,...,0.87,0.037,0.93,2.2e+06,0.48,0.014,0.49,0.24,6.5,0.27
111,1.1e+06,0,1.6e+04,1.2e+06,3.7e+05,1.4e+07,6.1e+05,1.5e+07,1.7e+07,10HHH5,...,0.87,0.036,0.93,2.1e+06,0.54,0.0074,0.54,0.17,6.8,0.28
112,7.6e+05,0,3.1e+04,7.9e+05,3.5e+05,1.4e+07,5.8e+05,1.5e+07,1.6e+07,2HHH5,...,0.89,0.038,0.95,1.7e+06,0.44,0.018,0.46,0.2,8,0.34
113,1.2e+06,0,8e+04,1.3e+06,6.7e+05,3.4e+07,5.9e+05,3.5e+07,3.6e+07,5HHH6,...,0.93,0.016,0.96,2.6e+06,0.48,0.031,0.51,0.26,13,0.23


In [9]:
df_formulacao1.query("holding_cost_end_products_perc2 > setup_cost_ingredients_perc2")

,setup_cost_end_products,production_cost_end_products,holding_cost_end_products,end_products_cost,setup_cost_ingredients,production_cost_ingredients,holding_cost_end_ingredients,ingredients_cost,objective_function,Instance,...,production_cost_ingredients_perc,holding_cost_end_ingredients_perc,ingredients_cost_perc,objective_function_without_ingredients,setup_cost_end_products_perc2,holding_cost_end_products_perc2,end_products_cost_perc2,setup_cost_ingredients_perc2,production_cost_ingredients_perc2,holding_cost_end_ingredients_perc2


% sem considerar o custo de produção dos ingredientes na fob

In [10]:
df_formulacao1.query("Instance == '5HHH2'")[cols2]

,setup_cost_end_products_perc2,holding_cost_end_products_perc2,setup_cost_ingredients_perc2,holding_cost_end_ingredients_perc2
0,0.35,0.025,0.42,0.2
102,0.45,0.031,0.27,0.25


In [11]:
#px.box(df_formulacao1, y="setup_cost_end_products_perc")

In [12]:
df_variaveis.head()

,Unnamed: 0,produto,periodo,variable,value,ingrediente,amount_of_end_products,instance
0,0,0,0,end_products,1.8e+03,NaN,1,2LLL1
1,1,0,1,end_products,0,NaN,1,2LLL1
2,2,0,2,end_products,2.1e+03,NaN,1,2LLL1
3,3,0,3,end_products,0,NaN,1,2LLL1
4,4,0,4,end_products,2e+03,NaN,1,2LLL1


Analisando as variáveis da instância 5HHH2.
- A chave da execução seria períodos-ingredientes-capacidade -> 30\*5*\*2=300 (máximo períodos-ingredientes-capacidade)
    - Não coloquei uma coluna de capacidade, então não foi possível segregar por capacidade.
- Ambas as instâncias com 1 e 2 produtos tem 270 períodos-ingredientes > 0
    - Sendo que a instância com 2 produtos, tem o dobro de estoque.
- Porém, ambas as instâncias fazem 20 períodos-ingredients de setup. (muito pouco em relação ao estoque).

In [13]:
df_variaveis_clean = df_variaveis.copy()
df_variaveis_clean['value'] = np.where(df_variaveis_clean["value"] < 0, 0, df_variaveis_clean['value'])
df_aux1 = df_variaveis_clean.query("instance == '5HHH2' and variable in ['inventory_ingredients']").copy()
df_aux1["inventory_ingredients_sign"] = np.sign(df_aux1['value'])
df_aux2 = df_variaveis.query("instance == '5HHH2' and variable in ['setup_ingredients']").copy()
df_aux3 = pd.concat([df_aux1, df_aux2])
df_aux3.drop_duplicates().query("instance == '5HHH2'").groupby(by=["instance", "variable", "amount_of_end_products"])[["inventory_ingredients_sign", "value"]].sum()

inventory_ingredients_sign  \
instance variable              amount_of_end_products                               
5HHH2    inventory_ingredients 1                                          2.7e+02   
                               2                                          2.7e+02   
         setup_ingredients     1                                                0   
                               2                                                0   

                                                        value  
instance variable              amount_of_end_products          
5HHH2    inventory_ingredients 1                      3.7e+05  
                               2                      7.5e+05  
         setup_ingredients     1                           15  
                               2                           15

O setup dos ingredientes é muito espaçado.

Período 0, depois período 14

In [14]:
df_variaveis.query("instance == '5HHH2' and variable == 'setup_ingredients' and value > 0")

,Unnamed: 0,produto,periodo,variable,value,ingrediente,amount_of_end_products,instance
113790,0,NaN,0,setup_ingredients,1,0,1,5HHH2
113804,14,NaN,14,setup_ingredients,1,0,1,5HHH2
113820,30,NaN,0,setup_ingredients,1,1,1,5HHH2
113834,44,NaN,14,setup_ingredients,1,1,1,5HHH2
113850,60,NaN,0,setup_ingredients,1,2,1,5HHH2
113864,74,NaN,14,setup_ingredients,1,2,1,5HHH2
113880,90,NaN,0,setup_ingredients,1,3,1,5HHH2
113894,104,NaN,14,setup_ingredients,1,3,1,5HHH2
113910,120,NaN,0,setup_ingredients,1,4,1,5HHH2
113924,134,NaN,14,setup_ingredients,1,4,1,5HHH2


Para os produtos finais, ocorre mais setups em relação aos ingredients, entretanto, não ocorre todos os períodos (a cada 2 períodos, segunda tabela abaixo).

Para 2 produtos, foi estocado mais produtos, como para os ingredientes (tabela acima), porém, diferente dos ingredients, a quantidade de setup aumentou, isso acontece pq a demanda aumentou, existe uma restrição de capacidade para produtos finais, mas não para ingredientes, logo foi possível com a mesma quantidade de setups, estocar mais ingredientes.

In [15]:
df_variaveis_clean = df_variaveis.copy()
df_variaveis_clean['value'] = np.where(df_variaveis_clean["value"] < 0, 0, df_variaveis_clean['value'])
df_aux1 = df_variaveis_clean.query("instance == '5HHH2' and variable in ['inventory_end_products']").copy()
df_aux1["inventory_end_products_sign"] = np.sign(df_aux1['value'])
df_aux2 = df_variaveis.query("instance == '5HHH2' and variable in ['setup_end_products']").copy()
df_aux3 = pd.concat([df_aux1, df_aux2])
df_aux3.drop_duplicates().query("instance == '5HHH2'").groupby(by=["instance", "variable", "amount_of_end_products"])[["inventory_end_products_sign", "value"]].sum()

inventory_end_products_sign  \
instance variable               amount_of_end_products                                
5HHH2    inventory_end_products 1                                                53   
                                2                                           1.1e+02   
         setup_end_products     1                                                 0   
                                2                                                 0   

                                                         value  
instance variable               amount_of_end_products          
5HHH2    inventory_end_products 1                      3.5e+04  
                                2                      7.1e+04  
         setup_end_products     1                           30  
                                2                           53

In [16]:
df_variaveis.query("instance == '5HHH2' and variable == 'setup_end_products' and value > 0")

,Unnamed: 0,produto,periodo,variable,value,ingrediente,amount_of_end_products,instance
113430,0,0,0,setup_end_products,1,NaN,1,5HHH2
113432,2,0,2,setup_end_products,1,NaN,1,5HHH2
113434,4,0,4,setup_end_products,1,NaN,1,5HHH2
113436,6,0,6,setup_end_products,1,NaN,1,5HHH2
113438,8,0,8,setup_end_products,1,NaN,1,5HHH2
...,...,...,...,...,...,...,...,...
115823,53,1,23,setup_end_products,1,NaN,2,5HHH2
115824,54,1,24,setup_end_products,1,NaN,2,5HHH2
115826,56,1,26,setup_end_products,1,NaN,2,5HHH2
115827,57,1,27,setup_end_products,1,NaN,2,5HHH2
